# Neural Bandit Algorithm Evaluation Framework

## Graduate Research Project
**AI & Quantum Computing Laboratory**  
**Rochester Institute of Technology**

---

## Research Framework Overview

This comprehensive evaluation framework provides rigorous analysis of neural bandit algorithms with clear categorical distinction between different operational environments:

- **Baseline Environment**: Optimal performance benchmark (Oracle)
- **Stochastic Environment**: Natural random failures and network noise
- **Adversarial Environment**: Strategic intelligent attacks and malicious targeting

## Primary Research Questions

1. **Algorithm Robustness**: How do neural bandit algorithms perform across different threat models?
2. **Comparative Analysis**: Which algorithms demonstrate superior performance in specific scenarios?
3. **Quantified Performance**: What are the exact degradation metrics under adversarial conditions?
4. **Theoretical Validation**: Do experimental results align with established regret bounds?

## Key Research Contributions

- **Systematic Environment Categorization**: Clear baseline/stochastic/adversarial taxonomy
- **Multi-Algorithm Comparative Testing**: Comprehensive evaluation across 6+ algorithms
- **Quantified Robustness Metrics**: Precise performance degradation measurements
- **Publication-Ready Analysis**: Academic-quality visualizations and statistical validation

## Evaluation Methodology

The framework implements standardized testing protocols across three distinct categories:
- **Baseline**: Oracle performance establishing theoretical upper bounds
- **Stochastic**: Random environmental perturbations modeling realistic conditions  
- **Adversarial**: Strategic attack scenarios simulating malicious interference

Each algorithm undergoes identical testing conditions enabling direct performance comparison and robustness quantification across all operational environments.

## Threat Model Classification Framework

### Systematic Environment Taxonomy

This research framework establishes precise categorical distinctions for quantum network evaluation environments, addressing previous ambiguity in threat model classification:

### Environmental Categories

| Environment | Implementation | Threat Characteristic | Research Application |
|-------------|----------------|----------------------|---------------------|
| **Baseline** | `none` | Deterministic optimal performance | Theoretical upper bound |
| **Stochastic** | `stochastic`/`random` | Natural random failures | Realistic network conditions |
| **Adversarial** | `markov` | Oblivious strategic attacks | Pattern-based targeting |
| **Adversarial** | `adaptive` | Responsive strategic attacks | Feedback-driven targeting |
| **Adversarial** | `onlineadaptive` | Real-time strategic attacks | Dynamic threat adaptation |

### Research Contribution

This framework addresses a critical gap in existing literature where random network failures were often conflated with intentional adversarial attacks. The systematic categorization enables:

- **Precise Robustness Quantification**: Exact performance degradation measurements across threat categories
- **Comparative Algorithm Analysis**: Direct performance comparison under identical threat conditions
- **Theoretical Validation**: Empirical verification of regret bounds across different adversarial models
- **Reproducible Research Standards**: Standardized evaluation protocols for quantum network algorithms

### Methodological Significance

Previous research often lacked clear distinction between stochastic and adversarial environments, limiting the ability to assess true algorithm robustness under intentional attacks versus natural network degradation. This framework provides the necessary precision for rigorous academic evaluation of quantum routing algorithms.

## Environment Setup & Library Installation

In [1]:
# ============================================================
# Setup: Quantum MAB Framework (Paper 7 QBGP Testbed)
# ============================================================

# --- Install Dependencies ---
!pip install -q torch torchvision numpy matplotlib seaborn pandas tqdm scipy scikit-learn pmdarima networkx

# --- Core Imports ---
import os, sys, gc, warnings, importlib, subprocess
import itertools
import numpy as np
import matplotlib.pyplot as plt
import torch
import networkx as nx
from pathlib import Path

warnings.filterwarnings('ignore')

# --- Path Setup ---
print(f"Current working directory: {os.getcwd().split('/')[-1]}")
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    project_dir = '/content/drive/MyDrive/GA-Work/hybrid_variable_framework/Dynamic_Routing_Eval_Framework'
    os.chdir(project_dir)
    print("Running in Google Colab")
except ImportError:
    print("Running locally (not in Colab)")
    # Navigate to the correct directory for local execution
    current_dir = os.getcwd()
    if 'GA-Work' in current_dir and 'Dynamic_Routing_Eval_Framework' not in current_dir:
        project_dir = os.path.join(current_dir, 'hybrid_variable_framework', 'Dynamic_Routing_Eval_Framework')
        if os.path.exists(project_dir):
            os.chdir(project_dir)
            print(f"Changed to project directory: {os.getcwd().split('/')[-1]}")

# Add necessary paths for daqr package discovery
sys.path.append(os.path.join(os.getcwd(), 'src'))
sys.path.append(os.getcwd())  # Add current directory to find daqr package

print(f"Now working from: {os.getcwd().split('/')[-1]}")

# --- Framework Verification ---
print("Framework dependencies installed successfully")
print(f"Python version: {sys.version.split()[0]}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"NetworkX version: {nx.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print("Quantum MAB Models Evaluation Framework - Ready for Paper 7 (QBGP) testing")

# --- Cleanup Script Execution ---
root = os.path.abspath("../..")
cleanup_script = os.path.join(root, "cleanup_state_duplicates.py")
if os.path.exists(cleanup_script):
    print(f"\n🚿 Running cleanup script at: {cleanup_script}\n")
    result = subprocess.run(["python3", cleanup_script], text=True, capture_output=True)
    print("===== CLEANUP STDOUT =====\n", result.stdout)
    print("===== CLEANUP STDERR =====\n", result.stderr)
else:
    print("⚠️ Cleanup script not found, skipping...")

# --- Deep Cleanup ---
def deep_cleanup():
    to_clear = ["oracle", "gneuralucb", "expneuralucb", "cpursuitneuralucb",
                "icpursuitneuralucb", "evaluator", "results"]
    for name in to_clear:
        if name in globals():
            obj = globals().get(name)
            if hasattr(obj, "cleanup"): obj.cleanup(verbose=False)
            globals().pop(name, None)
    gc.collect()
    torch.set_default_dtype(torch.float32)
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("✓ Deep cleanup complete (memory cleared)")

deep_cleanup()

# Ensure daqr package is discoverable
PARENT_DIR = os.path.abspath("..")
if PARENT_DIR not in sys.path:
    sys.path.insert(0, PARENT_DIR)
    
# Verify daqr package can be found
try:
    import daqr
    print("✓ daqr package found successfully")
except ImportError as e:
    print(f"❌ daqr package not found: {e}")
    print(f"Current directory: {os.getcwd()}")
    print(f"Python path includes: {[p for p in sys.path if 'GA-Work' in p or 'daqr' in p]}")
    print("Please ensure you're running from the Dynamic_Routing_Eval_Framework directory")

# --- Final Module Setup ---
from daqr.core.quantum_physics              import (MemoryNoiseModel, FullPaper2FidelityCalculator)
from daqr.evaluation                        import experiment_runner, multi_run_evaluator, visualizer
from daqr.config                            import experiment_config
from daqr.algorithms                        import base_bandit, neural_bandits, predictive_bandits
from daqr.core                              import network_environment, qubit_allocator
from daqr.config.experiment_config          import ExperimentConfiguration
from experiments                            import stochastic_evaluation
from daqr.core.network_environment          import *
from daqr.core.qubit_allocator              import *
from daqr.algorithms.base_bandit            import *
from daqr.algorithms.neural_bandits         import *
from daqr.algorithms.predictive_bandits     import *
from daqr.evaluation.multi_run_evaluator    import *
from daqr.evaluation.experiment_runner      import *

# ============================================================================
# PAPER-SPECIFIC IMPORTS
# ============================================================================
from daqr.core.topology_generator           import Paper2TopologyGenerator
from daqr.core.topology_generator           import Paper7ASTopologyGenerator  # 🆕 Paper 7
from daqr.core.topology_generator           import Paper12WaxmanTopologyGenerator
from daqr.core.quantum_physics              import FiberLossNoiseModel
from daqr.core.quantum_physics              import FusionNoiseModel, FusionFidelityCalculator
from daqr.core.quantum_physics              import Paper12RetryFidelityCalculator
from daqr.core.quantum_physics              import Paper7RewardFunction  # 🆕 Paper 7
from daqr.core                              import attack_strategy

print("✓ All modules reloaded successfully (Paper 7 environment ready)")

# --- Config & Model Setup ---
for module in [experiment_config, network_environment, qubit_allocator, attack_strategy, base_bandit, 
               neural_bandits, predictive_bandits, experiment_runner, multi_run_evaluator, visualizer, 
               stochastic_evaluation]:
    importlib.reload(module)

config = ExperimentConfiguration()
models = config.NEURAL_MODELS

# ============================================================================
# FRAMEWORK CONFIGURATION (Paper 7 Optimized)
# ============================================================================
FRAMEWORK_CONFIG = {
    'exp_num': 5,
    'test_mode': True,
    'base_frames': 4000,
    'frame_step': 2000,
    'models': models,
    'intensity': 0.25,
    'routing_strategy': 'fixed',
    'capacity': 10000,
    'main_env': 'stochastic',

    # Environment parameters
    'env_attrs': {
        'intensity': 0.25,
        'base_seed': 12345,
        'reproducible': True
    },

    'default': {
        'num_paths': 4,
        'total_qubits': 35,
        'min_qubits_per_route': 2,
        'exploration_bonus': 2.0,
        'epsilon': 1.0,
        'seed': 42
    },
    
    # 🎯 Paper #7 (Liu et al. 2024 - QBGP) - PRIMARY TESTBED
    'paper7': {
        # Topology Configuration
        'k': 5,                      # k-shortest paths per ISP pair
        'n_qisps': 3,                # Number of quantum ISP nodes
        'num_paths': 4,              # Total paths for framework compatibility
        'max_nodes': 50,             # AS subgraph size (30-80 for testing, None for full)
        'network_scale': 'small',    # 'small' (30-50), 'medium' (100-200), 'large' (full)
        'topology_path': '/Users/pitergarcia/DataScience/Semester4/GA-Work/hybrid_variable_framework/Dynamic_Routing_Eval_Framework/daqr/core/topology_data/as20000101.txt',
        
        # Framework Parameters
        'total_qubits': 35,
        'min_qubits_per_route': 2,
        'exploration_bonus': 2.0,
        
        # Context & Reward Configuration
        'use_context_rewards': True,      # Enable context-aware reward function
        'reward_mode': 'neg_hop',         # Options: 'neg_hop', 'neg_degree', 'neg_length'
        'use_synthetic': False,           # Force synthetic topology (ignore topology_path)
        
        # Topology Processing
        'largest_cc_only': True,          # Use largest connected component
        'relabel_to_int': True,           # Relabel nodes to integers
        
        # Synthetic Fallback (if topology_path fails or use_synthetic=True)
        'synthetic_kind': 'barabasi_albert',
        'synthetic_params': {
            'n': 50,                      # Number of nodes
            'm': 3                        # Edges per new node
        }
    },
    
    # Paper #2 (Huang et al. - Neural Bandit Work)
    'paper2': {
        # Topology & Paths
        'num_paths': 4,
        'num_total_qubits': 35,
        'dest_node': 14,
        'num_nodes': 15,
        'source_node': 1,
        
        # Base Physics
        'p_init': 0.00001,
        'total_qubits': 35,
        'f_attenuation': 0.05,
        
        # Stochastic Noise Parameters
        'p_BSM': 0.2,
        'p_GateErrors': 0.2,
        'p_depol': 0.1,

        # State Configuration
        'testbed': 'paper2',
        'initial_state': 'idle',
        'state_total_qubits': {'busy': 35, 'idle': 43},
        
        # Bandit Algorithm
        'exploration_bonus': 2.0,
        'min_qubits_per_route': 2,
        'transition_trigger': True,
        'paper2_transition_interval': 50,
        'entanglement_success_factor': 4000,
        
        # Paper2 Features
        'use_paper2_rewards': True,
        'swap_mode': 'async',
        'memory_T2': 5000,
        'gate_error_rate': 0.02,
        'swap_delay_per_link': 100,
        'use_gate_error': True,
        'use_memory_decay': True,
    },
    # Paper #12 (Wang et al. 2024 - QuARC)
    # Reference: Clayton et al., ICNP 2024 - "Efficient Routing on Quantum Networks using Adaptive Clustering"
    # Repository: https://github.com/cbclayton/clustered-quantum-routing
    'paper12': {
        'testbed': 'paper12',
        
        # ========== TOPOLOGY CONFIGURATION ==========
        'topology_type': 'waxman_qcast',
        
        # Network parameters (Paper 12 baseline)
        'n_nodes': 100,              # ✅ Baseline: 100 nodes
        'avg_degree': 6,             # ✅ E_d: Average node degree
        'entanglement_prob': 0.6,    # ✅ E_p: Paper 12 ORIGINAL value
        'fusion_prob': 0.9,          # ✅ q: Paper 12 ORIGINAL value
        
        # Waxman topology parameters
        'waxman_alpha': 0.4,
        'waxman_beta': 0.2,
        
        # ========== SIMULATION PARAMETERS ==========
        'num_sd_pairs': 10,          # ✅ nsd: Concurrent S-D pairs
        'epoch_length': 500,         # ✅ T_epoch: QuARC reconfiguration interval
        'total_timeslots': 5000,     # ✅ T: Baseline simulation length
        'cutoff': 1e9,               # Request timeout
        
        # ========== QUART PROTOCOL PARAMETERS ==========
        'split_constant': 3,         # ✅ k: Number of shortest paths
        'enable_clustering': True,   # ✅ Dynamic cluster merge/split
        'enable_secondary_fusions': True,  # ✅ Paper 12 fusion protocol
        'use_dynamic_thresholding': True,
        
        # ========== FRAMEWORK ADAPTATION ==========
        'num_paths': 4,              # For framework: paths for bandit arms
        'total_qubits': 100,         # Estimated: n_nodes * avg_qubits_per_node
        'exploration_bonus': 1.5,    # For bandit algorithms
        'min_qubits_per_route': 3,   # For framework allocation
        'use_fusion_rewards': True,  # Use entanglement success as reward
    }
}

# --- Test Scenarios ---
if FRAMEWORK_CONFIG['main_env'] == 'stochastic':
    test_scenarios = {
        'none': 'Baseline (Optimal Conditions)',
        'stochastic': 'Stochastic Random Failures',
        'markov': 'Markov Adversarial Attack',
        'adaptive': 'Adaptive Adversarial Attack',
        'onlineadaptive': 'Online Adaptive Attack'
    }
    evaluation_type = "STOCHASTIC-FOCUSED"
else:
    test_scenarios = {
        'stochastic': 'Stochastic (Natural Network Failures)',
        'adaptive': 'Adversarial (Strategic Attacks)'
    }
    evaluation_type = "COMPARATIVE"

Current working directory: GA-Work
Running locally (not in Colab)
Changed to project directory: Dynamic_Routing_Eval_Framework
Now working from: Dynamic_Routing_Eval_Framework
Framework dependencies installed successfully
Python version: 3.12.11
PyTorch version: 2.8.0
NumPy version: 1.26.4
NetworkX version: 3.5
Matplotlib version: 3.10.6
Quantum MAB Models Evaluation Framework - Ready for Paper 7 (QBGP) testing
⚠️ Cleanup script not found, skipping...
✓ Deep cleanup complete (memory cleared)
✓ daqr package found successfully
PyTorch version: 2.8.0
NumPy version: 1.26.4
Using device: cpu
PyTorch version: 2.8.0
NumPy version: 1.26.4
Using device: cpu
✓ All modules reloaded successfully (Paper 7 environment ready)
PyTorch version: 2.8.0
NumPy version: 1.26.4
Using device: cpu
PyTorch version: 2.8.0
NumPy version: 1.26.4
Using device: cpu

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL RE

## Comparative Analysis Framework: Stochastic versus Adversarial Environments

### Research Focus

This evaluation constitutes the primary empirical contribution of the research: systematic quantification of algorithm performance across fundamentally different operational conditions that distinguish between natural system failures and intentional strategic attacks.

### Environmental Characterization

**Stochastic Environment**
- **Operational Model**: Natural random failures representing realistic network degradation patterns
- **Attack Distribution**: Probabilistic failures following uniform random distribution
- **Research Significance**: Establishes baseline performance metrics under standard operational conditions

**Adversarial Environment**  
- **Operational Model**: Strategic intelligent attacks systematically targeting algorithmic decision-making processes
- **Attack Distribution**: Adaptive targeting mechanisms that dynamically respond to observed algorithm behavior
- **Research Significance**: Evaluates robustness under worst-case strategic threat scenarios

### Experimental Predictions

Based on the theoretical analysis and algorithm architecture, the following empirical outcomes are anticipated:

**Performance Superiority Hypothesis**
EXPNeuralUCB will demonstrate measurably superior performance retention in adversarial environments relative to baseline neural bandit algorithms lacking specialized adversarial robustness mechanisms.

**Bounded Degradation Hypothesis**
Performance degradation under adversarial conditions will remain within acceptable operational limits, specifically maintaining performance within 85% of stochastic environment baselines.

**Stability Hypothesis**
Algorithm performance rankings will exhibit stability across varying adversarial attack intensities, indicating consistent robustness characteristics rather than scenario-dependent performance fluctuations.

### Research Methodology

The comparative analysis employs identical experimental conditions across both environments, enabling precise quantification of performance degradation attributable to adversarial targeting while controlling for environmental variables and maintaining statistical rigor in the evaluation process.

In [2]:
# ============================================================================
# PAPER 12 (QUART) HELPER FUNCTIONS
# ============================================================================

def generate_paper12_paths(topology, num_paths: int, seed: int):
    """
    Generate random source-destination paths for Paper 12 (QuARC).
    
    Unlike Paper 7's ISP-based selection, Paper 12 uses random S-D pairs
    to evaluate adaptive clustering behavior across diverse routing scenarios.
    
    Args:
        topology: NetworkX graph (Waxman topology)
        num_paths: Number of paths to generate (typically 4 for framework testing)
        seed: Random seed
        
    Returns:
        List of paths (each path is a list of nodes)
    """
    rng = np.random.default_rng(seed)
    nodes = list(topology.nodes())
    paths = []
    attempts = 0
    max_attempts = 10 * num_paths
    
    while len(paths) < num_paths and attempts < max_attempts:
        attempts += 1
        src, dst = rng.choice(nodes, 2, replace=False)
        try:
            path = nx.shortest_path(topology, src, dst)
            if path not in paths:
                paths.append(path)
        except nx.NetworkXNoPath:
            continue
    
    if len(paths) < num_paths:
        raise RuntimeError(f"Could not find {num_paths} valid paths in Waxman topology")
    
    return paths


def generate_paper12_contexts(paths, topology, fusion_prob: float = 0.9):
    """
    Generate context vectors for Paper 12 (QuARC) paths.
    
    Context features match QuARC protocol state:
    - hop_count: Number of quantum hops (links) in path
    - normalized_avg_degree: Average node degree normalized by max degree
    - fusion_prob: Quantum fusion gate success probability
    
    Args:
        paths: List of paths (each path is a list of nodes)
        topology: NetworkX graph
        fusion_prob: Fusion gate success probability (q parameter)
        
    Returns:
        List of context arrays (one per path), each with shape [1, 3]
    """
    contexts = []
    degrees = dict(topology.degree())
    max_degree = max(degrees.values()) if degrees else 1.0
    
    for path in paths:
        # Feature 1: Hop count (number of quantum links)
        hop_count = float(len(path) - 1)
        
        # Feature 2: Average node degree (normalized)
        path_degrees = [degrees[n] for n in path]
        avg_degree = float(sum(path_degrees) / len(path_degrees)) if path_degrees else 0.0
        normalized_avg_degree = avg_degree / max_degree if max_degree > 0 else 0.0
        
        # Feature 3: Fusion probability (constant per configuration)
        fusion_prob_feature = float(fusion_prob)
        
        # Context vector: [hop_count, normalized_avg_degree, fusion_prob]
        context_vector = np.array([hop_count, normalized_avg_degree, fusion_prob_feature], dtype=float)
        contexts.append([context_vector])  # Wrap in list for framework compatibility
    
    return contexts


def get_paper12_thresholds(n_nodes: int):
    """
    Get merge/split thresholds for QuARC clustering (Paper 12).
    
    In Paper 12, thresholds are computed via interpolation based on network size.
    This is a simplified version that returns basic thresholds.
    
    For full Paper 12 alignment, use get_thresholds(n) from quarc/thresholding.py
    which computes dynamic thresholds via:
        scale_thresholds(n, 16**2, merge_th_16x16_ent_passing, split_th_16x16_ent_passing)
    
    Args:
        n_nodes: Number of nodes in network
        
    Returns:
        tuple: (merge_threshold_func, split_threshold_func)
    """
    # Baseline thresholds from Paper 12 Table 2 / recreate_figs.py
    # For n=100, merge_th ≈ 0.65, split_th ≈ 0.85 (approximate values)
    
    # Simplified linear interpolation for now
    base_merge = 0.65
    base_split = 0.85
    
    # Scale adjustment based on network size (larger networks = tighter thresholds)
    size_factor = min(1.0, 100.0 / max(n_nodes, 1))
    
    merge_th = base_merge * size_factor
    split_th = base_split * size_factor
    
    # Return functions for compatibility with protocol interface
    merge_threshold_func = lambda: merge_th
    split_threshold_func = lambda: split_th
    
    return merge_threshold_func, split_threshold_func


# ============================================================================
# PAPER 7 HELPER FUNCTIONS
# ============================================================================

def generate_paper7_paths(topology, k: int, n_qisps: int, seed: int):
    """
    Generate k-shortest paths between n_qisps quantum ISP nodes.
    
    Args:
        topology: NetworkX graph (AS-level topology)
        k: Number of shortest paths per ISP pair
        n_qisps: Number of quantum ISP nodes
        seed: Random seed
        
    Returns:
        List of paths (each path is a list of nodes)
    """
    rng = np.random.default_rng(seed)
    nodes = list(topology.nodes())

    if len(nodes) < n_qisps:
        raise ValueError(f"Topology has {len(nodes)} nodes, need {n_qisps} for ISPs")
    
    # Select ISP nodes (prefer high-degree nodes like real BGP)
    degrees = dict(topology.degree())
    sorted_nodes = sorted(nodes, key=lambda n: degrees[n], reverse=True)
    isp_nodes = sorted_nodes[:n_qisps]  # Take top-degree nodes
    
    all_paths = []
    for src, dst in itertools.combinations(isp_nodes, 2):
        try:
            path_generator = nx.shortest_simple_paths(topology, src, dst, weight='distance')
            paths = list(itertools.islice(path_generator, k))
            all_paths.extend(paths)
        except nx.NetworkXNoPath:
            continue
    
    if not all_paths:
        raise RuntimeError(f"Could not find any paths between {n_qisps} ISP nodes")
    
    return all_paths


def generate_paper7_contexts(paths, topology):
    """
    Generate context vectors for each path: [hop_count, avg_degree, path_length].
    
    Args:
        paths: List of paths (each path is a list of nodes)
        topology: NetworkX graph
        
    Returns:
        List of context arrays, one per path (shape: [1, 3])
    """
    contexts = []
    
    for path in paths:
        # Feature 1: Hop count (AS path length)
        hop_count = len(path) - 1
        
        # Feature 2: Average node degree (bottleneck indicator)
        degrees = [topology.degree(node) for node in path]
        avg_degree = sum(degrees) / len(degrees) if degrees else 0.0

        # Feature 3: Physical path length (sum of edge distances)
        path_length = 0.0
        for i in range(len(path) - 1):
            edge_data = topology.get_edge_data(path[i], path[i+1])
            path_length += edge_data.get('distance', 1.0)

        # Context vector: [hop_count, avg_degree, path_length]
        context_vector = np.array([hop_count, avg_degree, path_length], dtype=float)
        contexts.append([context_vector])  # Wrap in list for framework compatibility
    
    return contexts


def get_physics_params(
    physics_model: str = "default",
    current_frames: int = 4000,
    base_seed: int = 42,
    qubit_cap=None,
    *,
    topology: "nx.Graph | None" = None,
    topology_model: str | None = None,
    topology_path: str | Path | None = None,
    topology_max_nodes: int | None = None,
    topology_largest_cc_only: bool = True,
    topology_relabel_to_int: bool = True,
    synthetic_kind: str = "barabasi_albert",
    synthetic_params: dict | None = None,
):
    """
    Unified physics parameter generator for all testbeds.
    
    Returns:
        dict: {noise_model, fidelity_calculator, external_topology, 
               external_contexts, external_rewards}
    """
    
    # ============================================================================
    # 🎯 PAPER 7 (QBGP) - PRIMARY TESTBED
    # ============================================================================
    if physics_model == "paper7":
        import time
        start_time = time.time()
        
        paper7_cfg = FRAMEWORK_CONFIG['paper7']
        node_num = paper7_cfg.get('max_nodes')

        # --- Topology Generation ---
        if topology is not None:
            final_topology = topology
            print(f"📊 Paper7 Topology: User-provided ({len(topology.nodes())} nodes)")
        else:
            # Determine if using synthetic or real AS data
            if paper7_cfg.get('use_synthetic', False) or not paper7_cfg.get('topology_path'):
                # Synthetic fallback
                synth_params = synthetic_params or paper7_cfg.get('synthetic_params', {'n': 50, 'm': 3})
                topo_gen = Paper7ASTopologyGenerator(
                    edge_list_path="dummy_nonexistent.txt",
                    max_nodes=topology_max_nodes or node_num,
                    seed=base_seed,
                    synthetic_fallback=True,
                    synthetic_kind=paper7_cfg.get('synthetic_kind', 'barabasi_albert'),
                    synthetic_params=synth_params
                )
                print(f"📊 Paper7 Topology: Synthetic ({paper7_cfg.get('synthetic_kind')}, n={synth_params.get('n', 50)})")
            else:
                # Real AS topology
                topo_gen = Paper7ASTopologyGenerator(
                    edge_list_path=paper7_cfg['topology_path'],
                    max_nodes=node_num,
                    seed=base_seed,
                    relabel_to_integers=paper7_cfg.get('relabel_to_int', True),
                    largest_cc_only=paper7_cfg.get('largest_cc_only', True),
                    synthetic_fallback=True
                )
                topo_path_short = paper7_cfg['topology_path'].split('/')[-1]
                print(f"📊 Paper7 Topology: Real AS ({topo_path_short})")
            
            final_topology = topo_gen.generate()

        # --- Path Generation ---
        k = paper7_cfg["k"]
        n_qisps = paper7_cfg["n_qisps"]
        paths = generate_paper7_paths(final_topology, k, n_qisps, base_seed)
        contexts = generate_paper7_contexts(paths, final_topology)
        
        elapsed_ms = (time.time() - start_time) * 1000
        print(f"📊 Paper7 Paths: {len(paths)} paths from {k}-shortest between {n_qisps} ISPs")
        print(f"📊 Paper7 Contexts: {len(contexts)} context vectors generated")

        # --- Reward Function (Optional) ---
        external_rewards = None
        if paper7_cfg.get('use_context_rewards', False):
            reward_mode = paper7_cfg.get('reward_mode', 'neg_hop')
            reward_func = Paper7RewardFunction(mode=reward_mode)
            external_rewards = []
            for ctx_list in contexts:
                path_rewards = [reward_func.compute(ctx) for ctx in ctx_list]
                external_rewards.append(path_rewards)
            print(f"📊 Paper7 Rewards: Context-aware (mode={reward_mode})")
        else:
            print(f"📊 Paper7 Rewards: Using default framework rewards")

        print(f"⏱️  get_physics_params_paper7() time: {elapsed_ms:.1f} ms")
        
        return {
            "noise_model": None,
            "fidelity_calculator": None,
            "external_topology": final_topology,
            "external_contexts": contexts,
            "external_rewards": external_rewards
        }
    
    # ============================================================================
    # PAPER 2 (Huang et al.)
    # ============================================================================
    elif physics_model == "paper2":
        p2_config = FRAMEWORK_CONFIG["paper2"]
        topo_gen = Paper2TopologyGenerator(num_nodes=p2_config["num_nodes"], seed=base_seed)
        topo = topo_gen.generate()
        
        try:
            path_generator = nx.shortest_simple_paths(
                topo, p2_config["source_node"], p2_config["dest_node"], weight="distance"
            )
            paths = list(itertools.islice(path_generator, p2_config["num_paths"]))
        except nx.NetworkXNoPath:
            paths = [[p2_config["source_node"], p2_config["dest_node"]]] * p2_config["num_paths"]
        
        # Stochastic noise model
        noise_model = FiberLossNoiseModel(
            topology=topo,
            paths=paths,
            p_init=p2_config.get("p_init", 0.00001),
            f_attenuation=p2_config.get("f_attenuation", 0.05)
        )
        
        # Fidelity calculator with optional memory decay
        if p2_config.get('use_memory_decay', False):
            memory_model = MemoryNoiseModel(
                T2=p2_config.get("memory_T2", 5000),
                swap_delay_per_link=p2_config.get("swap_delay_per_link", 100)
            )
            if p2_config.get("swap_mode", "sync") == "sync":
                memory_model = None
        else:
            memory_model = None
        
        fidelity_calc = FullPaper2FidelityCalculator(
            gate_error_rate=p2_config.get("gate_error_rate", 0.02) if p2_config.get('use_gate_error', False) else 0.0,
            memory_model=memory_model
        )
        
        print(f"📊 Paper2 Physics: Fiber Loss + {'Memory Decay' if memory_model else 'No Memory'}")
        
        return {
            "noise_model": noise_model,
            "fidelity_calculator": fidelity_calc,
            "external_topology": topo,
            "external_contexts": None,
            "external_rewards": None
        }

    # ============================================================================
    # PAPER 12 (Wang et al. - QuARC)
    # ============================================================================
    # ============================================================================
    # PAPER 12 (Wang et al. - QuARC)
    # ============================================================================
    elif physics_model == 'paper12':
        p12config = FRAMEWORK_CONFIG['paper12']
        
        # Get base Paper12 physics
        physics_params = get_physics_params_paper12(p12config, seed=base_seed, qubit_cap=qubit_cap)
        base_fidelity_calc = physics_params['fidelity_calculator']
        
        # Optional retry logic (only if config provides it)
        retry_threshold = p12config.get('retry_threshold', None)
        if retry_threshold is not None:
            max_attempts = p12config.get('max_retry_attempts', 3)
            decay_rate = p12config.get('retry_decay_rate', 0.95)
            fidelity_calc = Paper12RetryFidelityCalculator(
                base_calculator=base_fidelity_calc,
                threshold=retry_threshold,
                max_attempts=max_attempts,
                decay_rate=decay_rate
            )
            physics_params['fidelity_calculator'] = fidelity_calc
            
            metadata = {
                'paper': 'Wang2024Paper12',
                'retry_enabled': True,
                'retry_threshold': retry_threshold,
                'max_attempts': max_attempts,
                'decay_rate': decay_rate,
            }
            physics_params['metadata'] = metadata
            print(f"📊 Paper12 Physics: Fusion (prob={p12config['fusion_prob']}) + Retry Logic")
        else:
            metadata = {
                'paper': 'Wang2024Paper12',
                'retry_enabled': False,
            }
            physics_params['metadata'] = metadata
            print(f"📊 Paper12 Physics: Fusion (prob={p12config['fusion_prob']})")
        
        return physics_params
    
    # ============================================================================
    # DEFAULT (No special physics)
    # ============================================================================
    else:
        return {
            "noise_model": None,
            "fidelity_calculator": None,
            "external_topology": topology,
            "external_contexts": None,
            "external_rewards": None
        }


def get_physics_params_paper12(config, seed, qubit_cap):
    """
    Paper #12 (QuARC) physics adapter - CORRECTED VERSION.
    
    Uses Paper 12 standard parameters:
    - E_p = 0.6 (entanglement probability)
    - q = 0.9 (fusion success probability)
    - k = 3 (split constant for Girvan-Newman)
    """
    
    # Generate Waxman topology
    topology = Paper12WaxmanTopologyGenerator().generate()
    
    # Generate 4 paths for framework testing
    num_paths = 4
    nodes = list(topology.nodes())
    rng = np.random.default_rng(seed)
    
    paths = []
    attempts = 0
    max_attempts = 10 * num_paths
    
    while len(paths) < num_paths and attempts < max_attempts:
        attempts += 1
        src, dst = rng.choice(nodes, 2, replace=False)
        try:
            path = nx.shortest_path(topology, src, dst)
            if path not in paths:
                paths.append(path)
        except nx.NetworkXNoPath:
            continue
    
    if len(paths) < num_paths:
        raise RuntimeError(f"Could not find {num_paths} valid paths")
    
    # ✅ CORRECT Paper 12 parameters
    fusion_prob = 0.9           # q from Paper 12
    entanglement_prob = 0.6     # E_p from Paper 12
    
    noise_model = FusionNoiseModel(
        topology=topology, 
        paths=paths, 
        fusion_prob=fusion_prob, 
        entanglement_prob=entanglement_prob
    )
    fidelity_calc = FusionFidelityCalculator()
    
    # ✅ CORRECT: Derive arms from topology
    external_contexts = []
    external_rewards = []
    degrees = dict(topology.degree())
    max_degree = max(degrees.values()) if degrees else 1.0
    
    for path in paths:
        # Get qubit capacity for this path
        min_qubits = min(topology.nodes[node].get('qubits', 5) for node in path)
        num_arms = min_qubits  # Or use k=3 for Paper 12 standard
        
        # Generate contexts
        hop_count = len(path) - 1
        path_degrees = [degrees[n] for n in path]
        avg_degree = float(sum(path_degrees) / len(path_degrees))
        f2_deg_norm = avg_degree / max_degree if max_degree > 0 else 0.0
        
        ctx = np.full((num_arms, 3), 
                     [float(hop_count), f2_deg_norm, fusion_prob], 
                     dtype=float)
        external_contexts.append(ctx)
        
        # ✅ CORRECT: Use actual entanglement physics (no Beta sampling)
        path_rewards = []
        for q_alloc in range(1, num_arms + 1):
            # Per-link success probability with q_alloc qubits
            p_link = 1 - (1 - entanglement_prob) ** q_alloc
            # End-to-end success (product over hops × fusion)
            p_path = (p_link ** hop_count) * fusion_prob
            # No ×100 scaling - use actual probability
            path_rewards.append(float(p_path))
        
        external_rewards.append(path_rewards)
    
    return {
        "external_topology": topology,
        "external_contexts": external_contexts,
        "external_rewards": external_rewards,
        "noise_model": noise_model,
        "fidelity_calculator": fidelity_calc,
    }



def force_release_resources(evaluator=None, verbose=True):
    """Force release of ALL resources that could block. Call AFTER each allocator completes."""
    cleanup_log = []

    # 1. Stop logging and close file handles
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs') and hasattr(evaluator.configs, 'backup_mgr'):
                backup_mgr = evaluator.configs.backup_mgr
                if hasattr(backup_mgr, 'stop_logging_redirect'):
                    backup_mgr.stop_logging_redirect()
                if hasattr(backup_mgr, '_log_file'):
                    try:
                        backup_mgr._log_file.close()
                    except:
                        pass
                if hasattr(backup_mgr, 'backup_registry'):
                    backup_mgr.backup_registry.clear()
            cleanup_log.append("✅ Backup manager cleaned")
        except Exception as e:
            cleanup_log.append(f"⚠️ Backup cleanup: {e}")

    # 2. Clear environment graphs
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs') and hasattr(evaluator.configs, 'environment'):
                env = evaluator.configs.environment
                if hasattr(env, 'topology') and hasattr(env.topology, 'clear'):
                    env.topology.clear()
                    del env.topology
                if hasattr(env, 'paths'):
                    env.paths = []
            cleanup_log.append("✅ Environment graphs cleared")
        except Exception as e:
            cleanup_log.append(f"⚠️ Environment cleanup: {e}")

    # 3. Break circular references
    if evaluator is not None:
        try:
            if hasattr(evaluator, 'configs'):
                if hasattr(evaluator.configs, 'backup_mgr'):
                    evaluator.configs.backup_mgr = None
                if hasattr(evaluator.configs, 'environment'):
                    evaluator.configs.environment = None
                evaluator.configs = None
            cleanup_log.append("✅ Circular references broken")
        except Exception as e:
            cleanup_log.append(f"⚠️ Reference cleanup: {e}")

    # 4. Clear model registries
    try:
        import sys
        for mod_name in list(sys.modules.keys()):
            if 'bandit' in mod_name.lower() or 'neural' in mod_name.lower():
                mod = sys.modules[mod_name]
                if hasattr(mod, '_model_registry'):
                    mod._model_registry.clear()
                if hasattr(mod, '_global_models'):
                    mod._global_models.clear()
        cleanup_log.append("✅ Model registries cleared")
    except Exception as e:
        cleanup_log.append(f"⚠️ Registry cleanup: {e}")

    # 5. Torch cleanup
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        cleanup_log.append("✅ Torch CUDA cleared")
    except Exception as e:
        cleanup_log.append(f"⚠️ Torch cleanup: {e}")

    # 6. Garbage collection
    collected = [gc.collect() for _ in range(3)]
    cleanup_log.append(f"✅ GC collected: {sum(collected)} objects")

    # 7. Close file descriptors
    try:
        import psutil
        process = psutil.Process()
        for f in process.open_files():
            if any(ext in f.path for ext in ['.pkl', '.log', '.csv']):
                try:
                    os.close(f.fd)
                except:
                    pass
        cleanup_log.append("✅ File descriptors closed")
    except Exception as e:
        cleanup_log.append(f"⚠️ FD cleanup: {e}")

    # 8. Delete evaluator and final collection
    if evaluator is not None:
        del evaluator
    gc.collect(2)

    if verbose:
        print("\n" + "="*70)
        print("🧹 FORCED RESOURCE RELEASE")
        print("="*70)
        for log in cleanup_log:
            print(log)
        print("="*70 + "\n")

    return True


# ============================================================================
# VALIDATION: Quick Paper 7 Sanity Check
# ============================================================================
print("\n" + "="*70)
print("🔍 PAPER 7 QUICK VALIDATION")
print("="*70)

try:
    # Test Paper 7 physics generation
    test_params = get_physics_params(
        physics_model='paper7',
        base_seed=42
    )
    
    print(f"✅ Topology: {len(test_params['external_topology'].nodes())} nodes, "
          f"{len(test_params['external_topology'].edges())} edges")
    print(f"✅ Contexts: {len(test_params['external_contexts'])} paths")
    print(f"✅ Rewards: {'Enabled' if test_params['external_rewards'] else 'Disabled'}")
    print("\n✓ Paper 7 integration validated successfully")
    
except Exception as e:
    print(f"❌ Validation failed: {e}")
    import traceback
    traceback.print_exc()

print("="*70 + "\n")

print("🚀 Ready to run Paper 7 (QBGP) experiments!")
print("   Example: PHYSICS_MODELS = ['paper7']")
print("            ALLOCATORS = ['ThompsonSampling', 'DynamicUCB']")

# ============================================================================
# VALIDATION: Quick Paper 12 Sanity Check
# ============================================================================
print("\n" + "="*70)
print("🔍 PAPER 12 (QuARC) QUICK VALIDATION")
print("="*70)

try:
    # Test Paper 12 physics generation
    test_params_p12 = get_physics_params(
        physics_model='paper12',
        base_seed=42
    )
    
    print(f"✅ Topology: {len(test_params_p12['external_topology'].nodes())} nodes, "
          f"{len(test_params_p12['external_topology'].edges())} edges")
    print(f"✅ Contexts: {len(test_params_p12['external_contexts'])} paths")
    print(f"✅ Context features per path: {test_params_p12['external_contexts'][0][0].shape}")
    
    # Print actual reward values
    print(f"✅ Rewards: {len(test_params_p12['external_rewards'])} reward lists")
    total_rew = sum(sum(r) for r in test_params_p12['external_rewards'])
    avg_rew = total_rew / sum(len(r) for r in test_params_p12['external_rewards'])
    print(f"   → Path 0 rewards (first 3): {[f'{r:.1f}' for r in test_params_p12['external_rewards'][0][:3]]}")
    print(f"   → Total aggregate reward: {total_rew:.1f}")
    print(f"   → Average per-arm reward: {avg_rew:.1f}")
    
    print(f"✅ Fusion probability: {FRAMEWORK_CONFIG['paper12']['fusion_prob']}")
    print(f"✅ Entanglement probability: {FRAMEWORK_CONFIG['paper12']['entanglement_prob']}")
    print(f"✅ Epoch length: {FRAMEWORK_CONFIG['paper12']['epoch_length']} timeslots")
    print(f"✅ Total simulation: {FRAMEWORK_CONFIG['paper12']['total_timeslots']} timeslots")
    print("\n✓ Paper 12 (QuARC) integration validated successfully")
    
except Exception as e:
    print(f"❌ Paper 12 validation failed: {e}")
    import traceback
    traceback.print_exc()


🔍 PAPER 7 QUICK VALIDATION
📊 Paper7 Topology: Real AS (as20000101.txt)
📊 Paper7 Paths: 15 paths from 5-shortest between 3 ISPs
📊 Paper7 Contexts: 15 context vectors generated
📊 Paper7 Rewards: Context-aware (mode=neg_hop)
⏱️  get_physics_params_paper7() time: 3.6 ms
✅ Topology: 50 nodes, 141 edges
✅ Contexts: 15 paths
✅ Rewards: Enabled

✓ Paper 7 integration validated successfully

🚀 Ready to run Paper 7 (QBGP) experiments!
   Example: PHYSICS_MODELS = ['paper7']
            ALLOCATORS = ['ThompsonSampling', 'DynamicUCB']

🔍 PAPER 12 (QuARC) QUICK VALIDATION
Generated paper12 Waxman topology: 100 nodes, 355 edges, avg degree: 7.10
📊 Paper12 Physics: Fusion (prob=0.9)
✅ Topology: 100 nodes, 355 edges
✅ Contexts: 4 paths
✅ Context features per path: (3,)
✅ Rewards: 4 reward lists
   → Path 0 rewards (first 3): ['0.3', '0.6', '0.8']
   → Total aggregate reward: 13.1
   → Average per-arm reward: 0.7
✅ Fusion probability: 0.9
✅ Entanglement probability: 0.6
✅ Epoch length: 500 timeslots
✅

In [3]:
# ============================================================
# Cell: Allocator + ExperimentConfiguration for Paper #12
# ALLOCATOR: Default (Baseline for Paper 12 QuARC)
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #12 (QuARC) - DEFAULT ALLOCATOR")
print("=" * 70)

# ============================================================================
# PAPER 12 EXECUTION FLOW:
# ============================================================================
# 1. get_physics_params(physics_model='paper12', ...):
#    → Calls get_physics_params_paper12(config, seed, qubit_cap)
#    → Generates Waxman topology (fallback) or BlobbedQCastNetwork (.net files)
#    → Calls generate_paper12_paths() to find 4 S-D paths
#    → Calls generate_paper12_contexts() to create context vectors
#    → Computes rewards via FusionNoiseModel (fusion_prob=0.9)
#
# 2. AllocatorRunner.run():
#    → Instantiates allocator (Default/Dynamic/ThompsonSampling/Random)
#    → Iterates through frame_step checkpoints (aligned to epoch_length=500)
#    → Each checkpoint: allocator.allocate() → select qubit distribution
#    → Rewards update via FusionNoiseModel.get_error_rates(path_idx)
#    → QuARC protocol checks thresholds at epoch boundaries
#
# 3. Results aggregated and saved:
#    → Metadata: paper=Wang2024Paper12, retry_enabled=False
#    → Topology: Waxman (n=100, E_d=6)
#    → Physics: E_p=0.6, q=0.9
#    → Simulation: T=1500ts, epoch=500, 3 epochs
# ============================================================================

# --- Paper 12 Allocator Configuration ---
ALLOCATORS = ["Default"]

# --- Paper 12 Run Parameters (Aligned with ICNP 2024 spec) ---
# Frame configuration aligned to QuARC epoch length
epoch_length_ts = FRAMEWORK_CONFIG['paper12']['epoch_length']  # 500 timeslots
total_sim_ts = FRAMEWORK_CONFIG['paper12']['total_timeslots']  # 5000 timeslots

attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = min(1500, total_sim_ts)  # 3 epochs for testing
frame_step          = epoch_length_ts           # Checkpoint every epoch
current_experiments = 1
last_backup         = True
base_cap            = False
overwrite           = False

RUNS                = [5]               # Acceptable for initial testing
SCALES              = [1, 1.5, 2]
ATTACK_SCENARIOS    = ['stochastic']    # Start simple
print(f"✅ Testing QuARC with {current_frames} timeslots")
print(f"✅ Expected reconfigurations: {current_frames // 500}")
print(f"✅ Statistical runs: {RUNS[0]}")

FRAMEWORK_CONFIG['exp_num']     = current_experiments
FRAMEWORK_CONFIG['base_frames'] = current_frames
FRAMEWORK_CONFIG['frame_step']  = frame_step

# Testbed Configuration
PHYSICS_MODELS = ['paper12']  # Paper 12 (QuARC)
ATTACK_SCENARIOS = ['stochastic']  # Start simple

print("\n" + "=" * 70)
print("PAPER 12 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocators:                 {ALLOCATORS}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)

for allocator_type in ALLOCATORS:
    print(f"\n{'='*70}")
    print(f"RUNNING: {allocator_type} on Paper 12 (QuARC)")
    print('='*70)

    for scale in SCALES:
        print(f"\n{'-'*70}")
        print(f"Preparing: {allocator_type} at scale {scale}")
        print(f"{'-'*70}")

        for physics_model in PHYSICS_MODELS:
            print(f"\n🔧 Generating physics parameters for model: {physics_model}") 
            try:
                # Create isolated runner instance
                custom_config = ExperimentConfiguration(
                    env_type=FRAMEWORK_CONFIG['main_env'],
                    scenarios=test_scenarios,
                    use_last_backup=last_backup,
                    models=models,
                    attack_intensity=attack_intensity,
                    scale=scale,
                    base_capacity=base_cap,
                    overwrite=overwrite
                )

                alloc_runner = AllocatorRunner(
                    allocator_type=allocator_type,
                    physics_models=[physics_model],
                    framework_config=FRAMEWORK_CONFIG,
                    scales=[scale],
                    runs=RUNS,
                    models=models,
                    test_scenarios=test_scenarios,
                    config=custom_config
                )

                # Run with Paper 12 physics
                alloc_runner.run(get_physics_params_func=get_physics_params)
                print(f"\n{allocator_type} COMPLETED SUCCESSFULLY")

            except Exception as e:
                print(f"\n{allocator_type} FAILED: {e}")
                import traceback
                traceback.print_exc()

print("\n" + "=" * 70)
print("ALL ALLOCATORS COMPLETE!")
print("=" * 70)

PAPER #12 (QuARC) - DEFAULT ALLOCATOR
✅ Testing QuARC with 1500 timeslots
✅ Expected reconfigurations: 3
✅ Statistical runs: 5

PAPER 12 ALLOCATOR EVALUATION
Allocators:                 ['Default']
Physics Models:             ['paper12']
Attack Scenarios:           ['stochastic']
Scales:                     [1, 1.5, 2]
Runs per Scale:             [5]
Total Frames:               1500

RUNNING: Default on Paper 12 (QuARC)

----------------------------------------------------------------------
Preparing: Default at scale 1
----------------------------------------------------------------------

🔧 Generating physics parameters for model: paper12

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL REGISTRY =====================

🔍 SCANNING LOCAL FILES
Parameters: load_to_drive=False, force=False

📂 Checking DRIVE mode: /content/drive/Shareddrives/ai_quantum_computing/quantum_data_lake
   ⚠️  Pa

In [4]:
# ============================================================
# Cell: Allocator + ExperimentConfiguration for Paper #12
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #12 (QuARC) QUANTUM ROUTING EVALUATION - ALL ALLOCATORS")
print("=" * 70)

# ------------------------------------------------------------
# 1) Allocator Selection
# ------------------------------------------------------------
ALLOCATORS = ["Dynamic"]

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
# CORRECTED Paper 12 Evaluation
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 1500      # 3 reconfiguration cycles
frame_step          = 500       # Epoch-aligned checkpoints
current_experiments = 1
last_backup         = True
base_cap            = False
overwrite           = False

RUNS                = [5]               # Acceptable for initial testing
SCALES              = [1, 1.5, 2]
ATTACK_SCENARIOS    = ['stochastic']    # Start simple
print(f"✅ Testing QuARC with {current_frames} timeslots")
print(f"✅ Expected reconfigurations: {current_frames // 500}")
print(f"✅ Statistical runs: {RUNS[0]}")

FRAMEWORK_CONFIG['exp_num']     = current_experiments
FRAMEWORK_CONFIG['base_frames'] = current_frames
FRAMEWORK_CONFIG['frame_step']  = frame_step

# Testbed Configuration
PHYSICS_MODELS = ['paper12']  # Paper 12 (QuARC)
ATTACK_SCENARIOS = ['stochastic']  # Start simple

print("\n" + "=" * 70)
print("PAPER 12 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocators:                 {ALLOCATORS}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)


for allocator_type in ALLOCATORS:
    print(f"\n{'='*70}")
    print(f"RUNNING: {allocator_type} on Paper 12 (QuARC)")
    print('='*70)

    for scale in SCALES:
        print(f"\n{'-'*70}")
        print(f"Preparing: {allocator_type} at scale {scale}")
        print(f"{'-'*70}")

        for physics_model in PHYSICS_MODELS:
            print(f"\n🔧 Generating physics parameters for model: {physics_model}") 
            try:
                # Create isolated runner instance
                custom_config = ExperimentConfiguration(
                    env_type=FRAMEWORK_CONFIG['main_env'],
                    scenarios=test_scenarios,
                    use_last_backup=last_backup,
                    models=models,
                    attack_intensity=attack_intensity,
                    scale=scale,
                    base_capacity=base_cap,
                    overwrite=overwrite
                )

                alloc_runner = AllocatorRunner(
                    allocator_type=allocator_type,
                    physics_models=[physics_model],
                    framework_config=FRAMEWORK_CONFIG,
                    scales=[scale],
                    runs=RUNS,
                    models=models,
                    test_scenarios=test_scenarios,
                    config=custom_config
                )

                # Run with Paper 12 physics
                alloc_runner.run(get_physics_params_func=get_physics_params)
                print(f"\n{allocator_type} COMPLETED SUCCESSFULLY")

            except Exception as e:
                print(f"\n{allocator_type} FAILED: {e}")
                import traceback
                traceback.print_exc()

print("\n" + "=" * 70)
print("ALL ALLOCATORS COMPLETE!")
print("=" * 70)

PAPER #12 (QuARC) QUANTUM ROUTING EVALUATION - ALL ALLOCATORS
✅ Testing QuARC with 1500 timeslots
✅ Expected reconfigurations: 3
✅ Statistical runs: 5

PAPER 12 ALLOCATOR EVALUATION
Allocators:                 ['Dynamic']
Physics Models:             ['paper12']
Attack Scenarios:           ['stochastic']
Scales:                     [1, 1.5, 2]
Runs per Scale:             [5]
Total Frames:               1500

RUNNING: Dynamic on Paper 12 (QuARC)

----------------------------------------------------------------------
Preparing: Dynamic at scale 1
----------------------------------------------------------------------

🔧 Generating physics parameters for model: paper12

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL REGISTRY =====================

🔍 SCANNING LOCAL FILES
Parameters: load_to_drive=False, force=False

📂 Checking DRIVE mode: /content/drive/Shareddrives/ai_quantum_computing/qua

In [ ]:
# ============================================================
# Cell: Allocator + ExperimentConfiguration for Paper #12
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #12 (QuARC) QUANTUM ROUTING EVALUATION - ALL ALLOCATORS")
print("=" * 70)

# ------------------------------------------------------------
# 1) Allocator Selection
# ------------------------------------------------------------
ALLOCATORS = ["ThompsonSampling"]

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
# CORRECTED Paper 12 Evaluation
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 1500      # 3 reconfiguration cycles
frame_step          = 500       # Epoch-aligned checkpoints
current_experiments = 1
last_backup         = True
base_cap            = False
overwrite           = False

RUNS                = [5]               # Acceptable for initial testing
SCALES              = [1, 1.5, 2]
ATTACK_SCENARIOS    = ['stochastic']    # Start simple
print(f"✅ Testing QuARC with {current_frames} timeslots")
print(f"✅ Expected reconfigurations: {current_frames // 500}")
print(f"✅ Statistical runs: {RUNS[0]}")

FRAMEWORK_CONFIG['exp_num']     = current_experiments
FRAMEWORK_CONFIG['base_frames'] = current_frames
FRAMEWORK_CONFIG['frame_step']  = frame_step


# Testbed Configuration
PHYSICS_MODELS = ['paper12']  # Paper 12 (QuARC)
ATTACK_SCENARIOS = ['stochastic']  # Start simple

print("\n" + "=" * 70)
print("PAPER 12 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocators:                 {ALLOCATORS}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)


for allocator_type in ALLOCATORS:
    print(f"\n{'='*70}")
    print(f"RUNNING: {allocator_type} on Paper 12 (QuARC)")
    print('='*70)

    for scale in SCALES:
        print(f"\n{'-'*70}")
        print(f"Preparing: {allocator_type} at scale {scale}")
        print(f"{'-'*70}")

        for physics_model in PHYSICS_MODELS:
            print(f"\n🔧 Generating physics parameters for model: {physics_model}") 
            try:
                # Create isolated runner instance
                custom_config = ExperimentConfiguration(
                    env_type=FRAMEWORK_CONFIG['main_env'],
                    scenarios=test_scenarios,
                    use_last_backup=last_backup,
                    models=models,
                    attack_intensity=attack_intensity,
                    scale=scale,
                    base_capacity=base_cap,
                    overwrite=overwrite
                )

                alloc_runner = AllocatorRunner(
                    allocator_type=allocator_type,
                    physics_models=[physics_model],
                    framework_config=FRAMEWORK_CONFIG,
                    scales=[scale],
                    runs=RUNS,
                    models=models,
                    test_scenarios=test_scenarios,
                    config=custom_config
                )

                # Run with Paper 12 physics
                alloc_runner.run(get_physics_params_func=get_physics_params)
                print(f"\n{allocator_type} COMPLETED SUCCESSFULLY")

            except Exception as e:
                print(f"\n{allocator_type} FAILED: {e}")
                import traceback
                traceback.print_exc()

print("\n" + "=" * 70)
print("ALL ALLOCATORS COMPLETE!")
print("=" * 70)


PAPER #12 (QuARC) QUANTUM ROUTING EVALUATION - ALL ALLOCATORS
✅ Testing QuARC with 1500 timeslots
✅ Expected reconfigurations: 3
✅ Statistical runs: 5

PAPER 12 ALLOCATOR EVALUATION
Allocators:                 ['ThompsonSampling']
Physics Models:             ['paper12']
Attack Scenarios:           ['stochastic']
Scales:                     [1, 1.5, 2]
Runs per Scale:             [5]
Total Frames:               1500

RUNNING: ThompsonSampling on Paper 12 (QuARC)

----------------------------------------------------------------------
Preparing: ThompsonSampling at scale 1
----------------------------------------------------------------------

🔧 Generating physics parameters for model: paper12

===================== GET LOCAL REGISTRY =====================
→ Skipping local cache (force=True or file missing)

===================== BUILD LOCAL REGISTRY =====================

🔍 SCANNING LOCAL FILES
Parameters: load_to_drive=False, force=False

📂 Checking DRIVE mode: /content/drive/Shareddriv

In [ ]:
# ============================================================
# Cell: Allocator + ExperimentConfiguration for Paper #12
# ============================================================
importlib.reload(qubit_allocator)
importlib.reload(experiment_config)
importlib.reload(multi_run_evaluator)

from daqr.core.quantum_physics              import *
from daqr.evaluation.allocator_runner       import AllocatorRunner
from daqr.evaluation.multi_run_evaluator    import MultiRunEvaluator
from daqr.config.experiment_config          import ExperimentConfiguration

print("=" * 70)
print("PAPER #12 (QuARC) QUANTUM ROUTING EVALUATION - ALL ALLOCATORS")
print("=" * 70)

# ------------------------------------------------------------
# 1) Allocator Selection
# ------------------------------------------------------------
ALLOCATORS = ["Random"]

# ------------------------------------------------------------
# 2) Run Parameters
# ------------------------------------------------------------
# CORRECTED Paper 12 Evaluation
attack_intensity    = FRAMEWORK_CONFIG['intensity']
current_frames      = 1500      # 3 reconfiguration cycles
frame_step          = 500       # Epoch-aligned checkpoints
current_experiments = 1
last_backup         = True
base_cap            = False
overwrite           = False

RUNS                = [5]               # Acceptable for initial testing
SCALES              = [1, 1.5, 2]
ATTACK_SCENARIOS    = ['stochastic']    # Start simple
print(f"✅ Testing QuARC with {current_frames} timeslots")
print(f"✅ Expected reconfigurations: {current_frames // 500}")
print(f"✅ Statistical runs: {RUNS[0]}")

FRAMEWORK_CONFIG['exp_num']     = current_experiments
FRAMEWORK_CONFIG['base_frames'] = current_frames
FRAMEWORK_CONFIG['frame_step']  = frame_step

# Testbed Configuration
PHYSICS_MODELS = ['paper12']  # Paper 12 (QuARC)
ATTACK_SCENARIOS = ['stochastic']  # Start simple

print("\n" + "=" * 70)
print("PAPER 12 ALLOCATOR EVALUATION")
print("=" * 70)
print(f"Allocators:                 {ALLOCATORS}")
print(f"Physics Models:             {PHYSICS_MODELS}")
print(f"Attack Scenarios:           {ATTACK_SCENARIOS}")
print(f"Scales:                     {SCALES}")
print(f"Runs per Scale:             {RUNS}")
print(f"Total Frames:               {current_frames}")
print("=" * 70)


for allocator_type in ALLOCATORS:
    print(f"\n{'='*70}")
    print(f"RUNNING: {allocator_type} on Paper 12 (QuARC)")
    print('='*70)

    for scale in SCALES:
        print(f"\n{'-'*70}")
        print(f"Preparing: {allocator_type} at scale {scale}")
        print(f"{'-'*70}")

        for physics_model in PHYSICS_MODELS:
            print(f"\n🔧 Generating physics parameters for model: {physics_model}") 
            try:
                # Create isolated runner instance
                custom_config = ExperimentConfiguration(
                    env_type=FRAMEWORK_CONFIG['main_env'],
                    scenarios=test_scenarios,
                    use_last_backup=last_backup,
                    models=models,
                    attack_intensity=attack_intensity,
                    scale=scale,
                    base_capacity=base_cap,
                    overwrite=overwrite
                )

                alloc_runner = AllocatorRunner(
                    allocator_type=allocator_type,
                    physics_models=[physics_model],
                    framework_config=FRAMEWORK_CONFIG,
                    scales=[scale],
                    runs=RUNS,
                    models=models,
                    test_scenarios=test_scenarios,
                    config=custom_config
                )

                # Run with Paper 12 physics
                alloc_runner.run(get_physics_params_func=get_physics_params)
                print(f"\n{allocator_type} COMPLETED SUCCESSFULLY")

            except Exception as e:
                print(f"\n{allocator_type} FAILED: {e}")
                import traceback
                traceback.print_exc()

print("\n" + "=" * 70)
print("ALL ALLOCATORS COMPLETE!")
print("=" * 70)

# Paper 12 (QuARC) - Clustered Quantum Routing

## Quantum Adaptive Routing Using Clusters (QuARC)

**Reference**: Clayton et al., 2024 - "Efficient Routing on Quantum Networks using Adaptive Clustering" (ICNP 2024)

**Repository**: https://github.com/cbclayton/clustered-quantum-routing

### Key Features

- **Adaptive Clustering**: Dynamic network partitioning into clusters (blobs) based on entanglement generation success
- **Merge/Split Protocol**: Clusters merge when success rate drops below threshold, split when exceeds threshold
- **Fusion Management**: Handles n-way quantum state fusion within clusters with noise modeling
- **Waxman Topology**: Realistic topological model with spatial characteristics
- **Adaptive Thresholding**: Merge/split thresholds scale dynamically with cluster size

### Implementation Strategy

Our framework integrates Paper12's QuARC concepts into the neural bandit testbed:

1. **Cluster Representation**: Map QuARC "blobs" to multi-armed bandit paths
2. **Fusion as Actions**: Different cluster sizes = different action arms per path
3. **Context Features**: [cluster_size, fusion_prob, avg_entanglement_rate]
4. **Reward Function**: Entanglement success rate normalized to [0, 1]
5. **Adaptive Allocation**: Neural bandits learn optimal cluster size selection per network state

### Testbed Parameters

- **Topology**: Waxman random geometric (100-800 nodes)
- **Edge Success Probability (E_p)**: 0.6 (60% quantum state fidelity)
- **Fusion Success Probability (q)**: 0.9 (90% fusion gate fidelity)
- **Average Degree (E_d)**: 6 neighbors per node
- **Channel Width**: 3 quantum channels per edge
- **Number of SD Pairs**: 10 source-destination pairs per evaluation
- **Qubits per Node**: Variable based on cluster configuration
- **Request Generation**: Random or bimodal distance distribution


## Multi-Environment Performance Analysis

### Complete Evaluation Matrix

This research extends beyond the primary stochastic-adversarial comparison to provide comprehensive algorithm assessment across the complete spectrum of operational environments, establishing a thorough empirical foundation for robustness evaluation.

### Environmental Test Framework

| Environment | Classification | Threat Characteristics | Analytical Purpose |
|-------------|---------------|----------------------|-------------------|
| `none` | Baseline | Deterministic optimal conditions | Theoretical performance ceiling |
| `stochastic` | Probabilistic | Uniform random failures | Standard operational baseline |
| `markov` | Adversarial | Memory-dependent strategic attacks | Oblivious adversarial model |
| `adaptive` | Adversarial | Feedback-driven strategic attacks | Responsive adversarial model |
| `onlineadaptive` | Adversarial | Real-time adaptive strategic attacks | Sophisticated adversarial model |

### Research Contributions

**Comprehensive Threat Model Coverage**
The evaluation framework addresses the complete spectrum of operational conditions, from optimal deterministic environments through increasingly sophisticated adversarial scenarios, providing unprecedented coverage of realistic deployment conditions.

**Graduated Adversarial Complexity Analysis**  
The systematic progression from oblivious to sophisticated adversarial models enables precise quantification of algorithm performance degradation as threat sophistication increases, revealing critical robustness thresholds.

**Cross-Environment Validation Protocol**
Consistent algorithm ranking across multiple environments validates robustness claims and identifies algorithms with stable performance characteristics independent of operational conditions.

**Empirical Robustness Quantification**
The multi-environment approach enables precise measurement of performance degradation rates, establishing quantitative robustness metrics that support theoretical predictions and practical deployment decisions.

### Methodological Significance

This comprehensive evaluation protocol addresses limitations in existing literature where algorithm assessment often focuses on narrow operational scenarios, providing the empirical foundation necessary for robust algorithm deployment in practical quantum network environments.